# ResNets

Hacemos una configuración de una red neuronal residual como la que mos visto en las transparencias de clase, y vemos que los resultados son mejores que si utilizamos este tipo de arquitectura sin los atajos entre capas.

Hacemos la preparación de datos de MNIST:

In [ ]:
import keras
from keras.datasets import mnist

batch_size = 128
num_classes = 10
epochs = 12

# Dimensiones de entrada de la imagen
img_rows, img_cols = 28, 28

# Los datos de training, reordenados y separados en train y test
(x_train, y_train), (x_test, y_test) = mnist.load_data()

X_train_images = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
X_test_images = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Importamos los diferentes tipos de capas. Las novedades aquí son dos:

- Utilizamos una nueva capa, `add`, que nos permite hacer la adición de la salida de dos capas previas.
- Utilizamos otra API de definición de redes neuronales de Keras, el **Functional API**.

Este API nos permite expresar una capa como una función que se aplica sobre una capa previa, y junto con la nueva capa `add` es la que permite configurar una arquitectura residual como se ve en el siguiente código:

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, add, Dense
from keras.models import Model

num_classes = 10
inputs = Input(shape=(28, 28, 1))
conv1_1 = Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')(inputs)
conv1_2 = Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')(conv1_1)
conv1_3 = Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')(conv1_2)
skip1 = add([conv1_1, conv1_3])
conv1_4 = Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')(skip1)
maxpool1 = MaxPooling2D(pool_size=(2, 2))(conv1_4)
conv2_1 = Conv2D(32, (3, 3), activation='relu', padding='same')(maxpool1)
conv2_2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2_1)
skip1 = add([maxpool1, conv2_2])
conv2_3 = Conv2D(32, (3, 3), activation='relu', padding='same')(skip1)
maxpool2 = MaxPooling2D(pool_size=(2, 2))(conv2_3)
flat = Flatten()(maxpool2)
dense = Dense(64, activation='relu')(flat)
predictions = Dense(num_classes, activation='softmax')(dense)

model = Model(inputs=inputs, outputs=predictions)

In [ ]:
model.summary()

Compilamos y entrenamos la red con los datos de MNIST ya correctamente preprocesados:

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
cnn_w_res = model.fit(X_train_images, y_train,
                       batch_size=batch_size, epochs=epochs, verbose=1, validation_split=.1)

Probamos con una arqutiectura distinta, en la que añadimos sólo un atajo en vez de los dos que teníamos previamente:

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten
from keras.models import Model

num_classes = 10
inputs = Input(shape=(28, 28, 1))
conv1_1 = Conv2D(32, (3, 3), activation='relu',
                 padding='same')(inputs)
conv1_2 = Conv2D(32, (3, 3), activation='relu',
                 padding='same')(conv1_1)
maxpool1 = MaxPooling2D(pool_size=(2, 2))(conv1_2)
conv2_1 = Conv2D(32, (3, 3), activation='relu',
                 padding='same')(maxpool1)
conv2_2 = Conv2D(32, (3, 3), activation='relu',
                 padding='same')(conv2_1)
skip2 = add([maxpool1, conv2_2])
maxpool2 = MaxPooling2D(pool_size=(2, 2))(skip2)
flat = Flatten()(maxpool2)
dense = Dense(64, activation='relu')(flat)
predictions = Dense(num_classes, activation='softmax')(dense)

model = Model(inputs=inputs, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
cnn_w_res = model.fit(X_train_images, y_train,
                       batch_size=128, epochs=10, verbose=1, validation_split=.1)